In [1]:
# Question: Advanced Data Cleaning with Multiple Issues
# Objective: Handle multiple issues in one dataset, including missing values, duplicates, and outliers.
# Description: Given a dataset with various data quality issues, employ multiple data cleaning techniques.


import pandas as pd
import numpy as np
from scipy import stats

# Sample dataset with multiple issues
data = {
    'Age': [25, 30, 35, 1000, 45, np.nan, 50, 30, 30],
    'Salary': [50000, 60000, None, 80000, 0, 100000, 110000, 60000, 60000],
    'Gender': ['Male', 'Female', 'Female', 'Male', None, 'Female', 'Male', 'Female', 'Female']
}

df = pd.DataFrame(data)

# Function to handle missing values
def handle_missing_values(df):
    for column in df.select_dtypes(include=[np.number]).columns:
        df[column] = df[column].fillna(df[column].mean())
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    return df

# Function to remove duplicates
def remove_duplicates(df):
    return df.drop_duplicates(keep='first')

# Function to handle outliers using Z-score

def handle_outliers_zscore(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        z_scores = np.abs(stats.zscore(df[col]))
        median = df[col].median()
        df.loc[z_scores > 3, col] = median
    return df

# Function to apply log transformation (only on positive values)
def apply_log_transformation(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
    return df

# Function to add a new feature

def add_age_to_salary_ratio(df):
    df['Age_to_Salary_Ratio'] = df.apply(
        lambda row: row['Age'] / row['Salary'] if row['Salary'] not in [0, None, np.nan] else np.nan,
        axis=1
    )
    return df

# Cleaning pipeline
def clean_data_pipeline(df):
    df = handle_missing_values(df)
    df = remove_duplicates(df)
    df = handle_outliers_zscore(df)
    df = apply_log_transformation(df)
    df = add_age_to_salary_ratio(df)
    return df

# Apply the pipeline
cleaned_df = clean_data_pipeline(df)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(cleaned_df)


Cleaned DataFrame:
        Age     Salary  Gender  Age_to_Salary_Ratio
0  3.218876  10.819778    Male             0.297499
1  3.401197  11.002100  Female             0.309141
2  3.555348  11.082143  Female             0.320818
3  6.907755  11.289782    Male             0.611859
4  3.806662   0.000000  Female                  NaN
5  5.047449  11.512925  Female             0.438416
6  3.912023  11.608236    Male             0.337004


/tmp/ipykernel_3882/2302542846.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
/tmp/ipykernel_3882/2302542846.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_to_Salary_Ratio'] = df.apply(


In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from scipy import stats

# Sample dataset with missing values, duplicates, and potential outliers
data = {
    'Age': [25, 30, 35, None, 45, None, 50, 1000],
    'Salary': [50000, 60000, None, 80000, None, 100000, 110000, 120000],
    'Gender': ['Male', 'Female', 'Female', 'Male', None, 'Female', 'Male', 'Male']
}

# Create DataFrame
df = pd.DataFrame(data)

# ---- Handling Missing Values ----
# Impute missing numerical values with KNN (K=2)
knn_imputer = KNNImputer(n_neighbors=2)
df[['Age', 'Salary']] = knn_imputer.fit_transform(df[['Age', 'Salary']])

# Impute missing categorical values with mode
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)

# ---- Handling Duplicates ----
# Remove duplicate rows
df.drop_duplicates(inplace=True)

# ---- Handling Outliers with Z-Scores ----
# Calculate Z-scores for numerical columns
z_scores = np.abs(stats.zscore(df[['Age', 'Salary']]))
outliers = (z_scores > 3)

# Replace outliers with the column mean (for simplicity)
df[['Age', 'Salary']] = df[['Age', 'Salary']].mask(outliers, df[['Age', 'Salary']].mean(), axis=1)

# ---- Log Transformation for Skewed Data ----
# Applying log transformation to Salary column
df['Salary'] = np.log(df['Salary'])

# ---- Feature Engineering ----
# Create a new feature called 'Age_to_Salary_Ratio' (Age / Salary)
df['Age_to_Salary_Ratio'] = df['Age'] / df['Salary']

# ---- Display the cleaned and transformed DataFrame ----
print("Cleaned and Transformed DataFrame:")
print(df)


Cleaned and Transformed DataFrame:
      Age     Salary  Gender  Age_to_Salary_Ratio
0    25.0  10.819778    Male             2.310583
1    30.0  11.002100  Female             2.726752
2    35.0  10.915088  Female             3.206570
3    27.5  11.289782    Male             2.435831
4    45.0  11.350407    Male             3.964616
5   525.0  11.512925  Female            45.600921
6    50.0  11.608236    Male             4.307287
7  1000.0  11.695247    Male            85.504821


In [3]:
# Question: Data Transformation Techniques
# Objective: Transform skewed data using log transformation.
# Description: Perform a log transformation to handle skewness in a dataset, which is particularly useful for
# certain machine learning models.



In [4]:
# Question: Feature Engineering by Creating New Features
# Objective: Create a new feature based on existing features to add predictive power.
# Description: Generate additional features from existing data to potentially improve the performance of
# prediction models.


import pandas as pd
import numpy as np
from scipy import stats

# Sample dataset with multiple issues
data = {
    'Age': [25, 30, 35, 1000, 45, np.nan, 50, 30, 30],
    'Salary': [50000, 60000, None, 80000, 0, 100000, 110000, 60000, 60000],
    'Gender': ['Male', 'Female', 'Female', 'Male', None, 'Female', 'Male', 'Female', 'Female']
}

df = pd.DataFrame(data)

# Function to handle missing values
def handle_missing_values(df):
    for column in df.select_dtypes(include=[np.number]).columns:
        df[column] = df[column].fillna(df[column].mean())
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    return df

# Function to remove duplicates
def remove_duplicates(df):
    return df.drop_duplicates(keep='first')

# Function to handle outliers using Z-score

def handle_outliers_zscore(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        z_scores = np.abs(stats.zscore(df[col]))
        median = df[col].median()
        df.loc[z_scores > 3, col] = median
    return df

# Function to apply log transformation (only on positive values)
def apply_log_transformation(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
    return df

# Function to add a new feature

def add_age_to_salary_ratio(df):
    df['Age_to_Salary_Ratio'] = df.apply(
        lambda row: row['Age'] / row['Salary'] if row['Salary'] not in [0, None, np.nan] else np.nan,
        axis=1
    )
    return df

# Cleaning pipeline
def clean_data_pipeline(df):
    df = handle_missing_values(df)
    df = remove_duplicates(df)
    df = handle_outliers_zscore(df)
    df = apply_log_transformation(df)
    df = add_age_to_salary_ratio(df)
    return df

# Apply the pipeline
cleaned_df = clean_data_pipeline(df)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(cleaned_df)


Cleaned DataFrame:
        Age     Salary  Gender  Age_to_Salary_Ratio
0  3.218876  10.819778    Male             0.297499
1  3.401197  11.002100  Female             0.309141
2  3.555348  11.082143  Female             0.320818
3  6.907755  11.289782    Male             0.611859
4  3.806662   0.000000  Female                  NaN
5  5.047449  11.512925  Female             0.438416
6  3.912023  11.608236    Male             0.337004


/tmp/ipykernel_3882/3428310238.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
/tmp/ipykernel_3882/3428310238.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_to_Salary_Ratio'] = df.apply(


In [5]:
# Question: Handling Complex Outliers with Z-Scores
# Objective: Detect and handle outliers using Z-score method.
# Description: Use the Z-score method to identify outliers which significantly differ from the rest of the data points.

import pandas as pd
import numpy as np
from scipy import stats

# Sample dataset with multiple issues
data = {
    'Age': [25, 30, 35, 1000, 45, np.nan, 50, 30, 30],
    'Salary': [50000, 60000, None, 80000, 0, 100000, 110000, 60000, 60000],
    'Gender': ['Male', 'Female', 'Female', 'Male', None, 'Female', 'Male', 'Female', 'Female']
}

df = pd.DataFrame(data)

# Function to handle missing values
def handle_missing_values(df):
    for column in df.select_dtypes(include=[np.number]).columns:
        df[column] = df[column].fillna(df[column].mean())
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    return df

# Function to remove duplicates
def remove_duplicates(df):
    return df.drop_duplicates(keep='first')

# Function to handle outliers using Z-score

def handle_outliers_zscore(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        z_scores = np.abs(stats.zscore(df[col]))
        median = df[col].median()
        df.loc[z_scores > 3, col] = median
    return df

# Function to apply log transformation (only on positive values)
def apply_log_transformation(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
    return df

# Function to add a new feature

def add_age_to_salary_ratio(df):
    df['Age_to_Salary_Ratio'] = df.apply(
        lambda row: row['Age'] / row['Salary'] if row['Salary'] not in [0, None, np.nan] else np.nan,
        axis=1
    )
    return df

# Cleaning pipeline
def clean_data_pipeline(df):
    df = handle_missing_values(df)
    df = remove_duplicates(df)
    df = handle_outliers_zscore(df)
    df = apply_log_transformation(df)
    df = add_age_to_salary_ratio(df)
    return df

# Apply the pipeline
cleaned_df = clean_data_pipeline(df)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(cleaned_df)



Cleaned DataFrame:
        Age     Salary  Gender  Age_to_Salary_Ratio
0  3.218876  10.819778    Male             0.297499
1  3.401197  11.002100  Female             0.309141
2  3.555348  11.082143  Female             0.320818
3  6.907755  11.289782    Male             0.611859
4  3.806662   0.000000  Female                  NaN
5  5.047449  11.512925  Female             0.438416
6  3.912023  11.608236    Male             0.337004


/tmp/ipykernel_3882/306163412.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
/tmp/ipykernel_3882/306163412.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_to_Salary_Ratio'] = df.apply(


In [6]:
# Question: Data Imputation with K-Nearest Neighbors (KNN)
# Objective: Impute missing numerical values using the KNN method.
# Description: Use the K-nearest neighbors algorithm to fill in missing values, which considers the values of
# nearest neighbors for imputation.
import pandas as pd
import numpy as np
from scipy import stats

# Sample dataset with multiple issues
data = {
    'Age': [25, 30, 35, 1000, 45, np.nan, 50, 30, 30],
    'Salary': [50000, 60000, None, 80000, 0, 100000, 110000, 60000, 60000],
    'Gender': ['Male', 'Female', 'Female', 'Male', None, 'Female', 'Male', 'Female', 'Female']
}

df = pd.DataFrame(data)

# Function to handle missing values
def handle_missing_values(df):
    for column in df.select_dtypes(include=[np.number]).columns:
        df[column] = df[column].fillna(df[column].mean())
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    return df

# Function to remove duplicates
def remove_duplicates(df):
    return df.drop_duplicates(keep='first')

# Function to handle outliers using Z-score

def handle_outliers_zscore(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        z_scores = np.abs(stats.zscore(df[col]))
        median = df[col].median()
        df.loc[z_scores > 3, col] = median
    return df

# Function to apply log transformation (only on positive values)
def apply_log_transformation(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
    return df

# Function to add a new feature

def add_age_to_salary_ratio(df):
    df['Age_to_Salary_Ratio'] = df.apply(
        lambda row: row['Age'] / row['Salary'] if row['Salary'] not in [0, None, np.nan] else np.nan,
        axis=1
    )
    return df

# Cleaning pipeline
def clean_data_pipeline(df):
    df = handle_missing_values(df)
    df = remove_duplicates(df)
    df = handle_outliers_zscore(df)
    df = apply_log_transformation(df)
    df = add_age_to_salary_ratio(df)
    return df

# Apply the pipeline
cleaned_df = clean_data_pipeline(df)

# Display the cleaned DataFrame
print("Cleaned DataFrame:")
print(cleaned_df)




Cleaned DataFrame:
        Age     Salary  Gender  Age_to_Salary_Ratio
0  3.218876  10.819778    Male             0.297499
1  3.401197  11.002100  Female             0.309141
2  3.555348  11.082143  Female             0.320818
3  6.907755  11.289782    Male             0.611859
4  3.806662   0.000000  Female                  NaN
5  5.047449  11.512925  Female             0.438416
6  3.912023  11.608236    Male             0.337004


/tmp/ipykernel_3882/1832039676.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: np.log(x) if x > 0 else x)
/tmp/ipykernel_3882/1832039676.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age_to_Salary_Ratio'] = df.apply(
